<a href="https://colab.research.google.com/github/ashfaq-polit/Automation-Codes/blob/main/Automate%20Emails%20Daily.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import smtplib
import ssl
import os
from email.message import EmailMessage
from email.utils import formataddr
from mimetypes import guess_type
import getpass

def send_email(sender_email: str,
               sender_name: str,
               password:str,
               receiver_emails: list,
               email_body: str,
               email_subject: str = "No subject",
               is_html: bool = False,
               attachments: list = None) -> None:

    msg = EmailMessage()
    msg["Subject"] = email_subject
    msg["From"] = formataddr((f"{sender_name}", f"{sender_email}"))
    msg["BCC"] = sender_email  # Can add CC or BCC here if needed

    # Support both plain text and HTML emails
    if is_html:
        msg.add_alternative(email_body, subtype='html')
    else:
        msg.set_content(email_body)

    # Add attachments if provided
    if attachments:
        for file_path in attachments:
            try:
                with open(file_path, 'rb') as file:
                    file_data = file.read()
                    file_name = os.path.basename(file_path)
                    mime_type, _ = guess_type(file_path)
                    if mime_type:
                        mime_main, mime_subtype = mime_type.split('/')
                    else:
                        mime_main, mime_subtype = 'application', 'octet-stream'

                    msg.add_attachment(file_data, maintype=mime_main, subtype=mime_subtype, filename=file_name)
                    print(f"Attached file: {file_name}")
            except Exception as e:
                print(f"Failed to attach {file_path}: {e}")

    smtp_port = 587
    smtp_server = "smtp.gmail.com"

    # Adding ssl layer of security
    ssl_context = ssl.create_default_context()

    try:
        # Creating smtp server
        print("Connecting to Server...")
        my_server = smtplib.SMTP(smtp_server, smtp_port)
        my_server.starttls(context=ssl_context)

        # Login to smtp server
        my_server.login(sender_email, password)
        print("Connected to server!")

        # Sending email
        print(f"Sending email from: {sender_email}")
        print("**************************************")
        for receiver in receiver_emails:
            msg["To"] = receiver
            print(f"Sending email to: {receiver}")
            my_server.sendmail(sender_email, receiver, msg.as_string())
            print(f"...\nSuccessfully sent to: {receiver}")
            print("**************************************")
            del msg["To"]
    except Exception as e:
        print(f"ERROR: {e}")
    finally:
        my_server.quit()

# Example usage
sender_email = "ashfaq.polit@gmail.com"
sender_name = "Ashfaq Ahmed"

# Try to get password from env, else prompt securely
password = os.environ.get("EMAIL_PASSWORD")
if not password:
    print("EMAIL_PASSWORD environment variable not set.")
    password = getpass.getpass("Enter your Gmail App Password (input hidden): ")

# Generate an App Password:

# After enabling 2-Step Verification, go to: https://myaccount.google.com/apppasswords

# Sign in and select:

# App: "Mail"

# Device: Choose "Other" and name it (e.g., "Python Script")

# Click Generate.

# You’ll get a 16-character password (e.g., abcd efgh ijkl mnop)

email_subject = "Good morning"
email_body = """
<h1>Good Morning!</h1>
<p>Hope you have a <strong>wonderful day</strong>.</p>
"""
receiver_emails = ["ashfaq.ahmed026@outlook.com"]
attachments = ["/content/Checklist - T&D Equipment Returns for Remote Employees.docx", "/content/Leetcode 75 Questions (NeetCode on yt).xlsx"]

# Sending the email as HTML with attachments
send_email(sender_email, sender_name, password, receiver_emails, email_body, email_subject, is_html=True, attachments=attachments)

EMAIL_PASSWORD environment variable not set.
Enter your Gmail App Password (input hidden): ··········
Attached file: Checklist - T&D Equipment Returns for Remote Employees.docx
Attached file: Leetcode 75 Questions (NeetCode on yt).xlsx
Connecting to Server...
Connected to server!
Sending email from: ashfaq.polit@gmail.com
**************************************
Sending email to: ashfaq.ahmed026@outlook.com
...
Successfully sent to: ashfaq.ahmed026@outlook.com
**************************************
